In [7]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [8]:
concrete_train = pd.read_csv('concrete_train.csv')
concrete_test = pd.read_csv('concrete_test.csv')

In [9]:
concrete_train.head()

,Cement,Blast_Furnace_Slag,Fly_Ash,Water,Superplasticizer,Coarse_Aggregate,Fine_Aggregate,Age,Water_Cement_Ratio,Strength
0,251.4,0.0,118.3,188.5,6.4,1028.4,757.7,56,0.749801,36.64
1,339.0,0.0,0.0,197.0,0.0,968.0,781.0,7,0.581121,20.97
2,250.0,0.0,95.7,187.4,5.5,956.9,861.2,56,0.749600,38.33
3,233.8,0.0,94.6,197.9,4.6,947.0,852.2,100,0.846450,34.56
4,350.0,0.0,0.0,186.0,0.0,1050.0,770.0,28,0.531429,34.29


In [15]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

def VIF(df, columns):
    values = sm.add_constant(df[columns]).values
    num_columns = len(columns)+1
    vif = [variance_inflation_factor(values, i) for i in range(num_columns)]
    return pd.Series(vif[1:], index=columns)

First, I will train our model using the R-style formulas.

In [29]:
model1= smf.ols(formula='Strength ~ Cement + Blast_Furnace_Slag + Fly_Ash + Water + Superplasticizer + Coarse_Aggregate + Fine_Aggregate + Age + Water_Cement_Ratio', 
                 data=concrete_train).fit()
print(model1.summary())
cols = ['Cement', 'Blast_Furnace_Slag', 'Fly_Ash', 'Water', 'Superplasticizer', 'Coarse_Aggregate', 'Fine_Aggregate', 'Age', 'Water_Cement_Ratio']
VIF(concrete_train, cols)

                            OLS Regression Results                            
Dep. Variable:               Strength   R-squared:                       0.603
Model:                            OLS   Adj. R-squared:                  0.598
Method:                 Least Squares   F-statistic:                     119.9
Date:                Thu, 21 Oct 2021   Prob (F-statistic):          3.55e-136
Time:                        18:00:12   Log-Likelihood:                -2705.3
No. Observations:                 721   AIC:                             5431.
Df Residuals:                     711   BIC:                             5476.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept            -31.5803     32

/opt/conda/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


Cement                12.594748
Blast_Furnace_Slag     7.480888
Fly_Ash                6.089445
Water                  7.596369
Superplasticizer       2.937475
Coarse_Aggregate       5.212360
Fine_Aggregate         7.310910
Age                    1.110141
Water_Cement_Ratio     6.669677
dtype: float64

In order to narrow down our model, we can look at the VIFs or the p-values to help me select which independent variables I may need to remove. To narrow down the model, I should remove variables with VIFs > 5 or insignificant variables with p-values > 0.05.

Based on the VIFs above, cement seems problematic since its VIF > 10. However, our p-values indicate that course aggregate, fine aggregate, and water to cement ratio are insignificant. I will remove variables one by one to observe the effect on the R-squared and adjusted r-squared value. We would like to maximize our r-squared value while still narrowing down our model. 

In [20]:
model2 = smf.ols(formula='Strength ~  Blast_Furnace_Slag + Fly_Ash + Water + Superplasticizer + Coarse_Aggregate + Fine_Aggregate + Age + Water_Cement_Ratio', 
                 data=concrete_train).fit()
print(model2.summary())
cols_2 = ['Blast_Furnace_Slag', 'Fly_Ash', 'Water', 'Superplasticizer', 'Coarse_Aggregate', 'Fine_Aggregate', 'Age', 'Water_Cement_Ratio']
VIF(concrete_train, cols_2)

                            OLS Regression Results                            
Dep. Variable:               Strength   R-squared:                       0.560
Model:                            OLS   Adj. R-squared:                  0.555
Method:                 Least Squares   F-statistic:                     113.1
Date:                Thu, 21 Oct 2021   Prob (F-statistic):          2.12e-121
Time:                        17:53:55   Log-Likelihood:                -2742.5
No. Observations:                 721   AIC:                             5503.
Df Residuals:                     712   BIC:                             5544.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept            184.1306     22

/opt/conda/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


Blast_Furnace_Slag    4.299342
Fly_Ash               3.111559
Water                 6.169571
Superplasticizer      2.923183
Coarse_Aggregate      2.962581
Fine_Aggregate        3.858057
Age                   1.103416
Water_Cement_Ratio    4.115372
dtype: float64

Water has a high VIF (greater than 5), so I will remove water next.

In [23]:
model3 = smf.ols(formula='Strength ~  Blast_Furnace_Slag + Fly_Ash + Superplasticizer + Coarse_Aggregate + Fine_Aggregate + Age + Water_Cement_Ratio', 
                 data=concrete_train).fit()
print(model3.summary())
cols_3 = ['Blast_Furnace_Slag', 'Fly_Ash', 'Superplasticizer', 'Coarse_Aggregate', 'Fine_Aggregate', 'Age', 'Water_Cement_Ratio']
VIF(concrete_train, cols_3)

                            OLS Regression Results                            
Dep. Variable:               Strength   R-squared:                       0.535
Model:                            OLS   Adj. R-squared:                  0.531
Method:                 Least Squares   F-statistic:                     117.4
Date:                Thu, 21 Oct 2021   Prob (F-statistic):          3.05e-114
Time:                        17:55:24   Log-Likelihood:                -2761.8
No. Observations:                 721   AIC:                             5540.
Df Residuals:                     713   BIC:                             5576.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             52.8025      8

/opt/conda/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


Blast_Furnace_Slag    2.799356
Fly_Ash               2.372857
Superplasticizer      1.995456
Coarse_Aggregate      1.287210
Fine_Aggregate        1.596583
Age                   1.081394
Water_Cement_Ratio    2.349018
dtype: float64

All of the VIFs are now below 5, which is good. However, there are 2 variables -- coarse aggregate and fine aggregate, that have high p-values (greater than 0.05). I will remove these as well. 

In [33]:
model4 = smf.ols(formula='Strength ~  Blast_Furnace_Slag + Fly_Ash + Superplasticizer  + Age + Water_Cement_Ratio', 
                 data=concrete_train).fit()
print(model4.summary())
cols_4 = ['Blast_Furnace_Slag', 'Fly_Ash', 'Superplasticizer', 'Age', 'Water_Cement_Ratio']
VIF(concrete_train, cols_4)

                            OLS Regression Results                            
Dep. Variable:               Strength   R-squared:                       0.534
Model:                            OLS   Adj. R-squared:                  0.531
Method:                 Least Squares   F-statistic:                     164.2
Date:                Thu, 21 Oct 2021   Prob (F-statistic):          4.00e-116
Time:                        18:20:07   Log-Likelihood:                -2762.5
No. Observations:                 721   AIC:                             5537.
Df Residuals:                     715   BIC:                             5565.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             44.3554      1

/opt/conda/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


Blast_Furnace_Slag    1.793526
Fly_Ash               2.032596
Superplasticizer      1.646950
Age                   1.047508
Water_Cement_Ratio    1.848562
dtype: float64

Although I removed high VIFs one by one and then removed variables with high p-values, the resulting model has an R^2 and adjusted R^2 that is quite a bit lower than the original model1. I will try a different approach, where I remove the variables with high p-values first, and compare the two models on the train and test sets. Looking back at model1, which included all of the variables, coarse aggregate, fine aggregate, and water to cement ratio had high p-values. I will remove these 3 variables from the original model. 

In [34]:
model2_2= smf.ols(formula='Strength ~ Cement + Blast_Furnace_Slag + Fly_Ash + Water + Superplasticizer + Age', 
                 data=concrete_train).fit()
print(model2_2.summary())
cols_5 = ['Cement', 'Blast_Furnace_Slag', 'Fly_Ash', 'Water', 'Superplasticizer', 'Age']
VIF(concrete_train, cols_5)

                            OLS Regression Results                            
Dep. Variable:               Strength   R-squared:                       0.600
Model:                            OLS   Adj. R-squared:                  0.597
Method:                 Least Squares   F-statistic:                     178.7
Date:                Thu, 21 Oct 2021   Prob (F-statistic):          1.50e-138
Time:                        18:21:18   Log-Likelihood:                -2707.5
No. Observations:                 721   AIC:                             5429.
Df Residuals:                     714   BIC:                             5461.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             25.9785      5

/opt/conda/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


Cement                1.848406
Blast_Furnace_Slag    1.752002
Fly_Ash               2.299993
Water                 1.933888
Superplasticizer      2.397385
Age                   1.097979
dtype: float64

In [27]:
# compute out-of-sample R-squared using the test set
def OSR2(model, df_train, df_test, dependent_var):   
    y_test = df_test[dependent_var]
    y_pred = model.predict(df_test)
    SSE = np.sum((y_test - y_pred)**2)
    SST = np.sum((y_test - np.mean(df_train[dependent_var]))**2)    
    return 1 - (SSE/SST)

In [28]:
OSR2(model4, concrete_train, concrete_test, 'Strength') #removed Cement, Water, Coarse Aggregate, Fine Aggregate

0.6240291836899139

In [32]:
OSR2(model2_2, concrete_train, concrete_test, 'Strength') #removed Coarse Aggregate, Fine Aggregate, Water_Cement_Ratio


0.6340412339529531

Overall, I tried two overarching approaches and compared them. First, I decided to remove all variables (one by one) that had VIF values greater than 5, starting from the largest VIF value and working down. For model2, I removed cement, which had a VIF > 10. I then removed water, which had a VIF > 5. After that, all of my remaining VIFs were below 5, so I removed insignificant variables with p-values > 0.05, like coarse aggregate and fine aggregate, which was my model4. This ended up reducing the R^2 by a lot; it decreased from 0.603 in model1 to 0.534 in model4, which indicates that the overall quality of the model decreased and that model4 is a worse fit to the training data. I then used OSR^2 to evaluate the model on the test set. The OSR^2 value for model4 was 0.624. 

In my second approach, I decided to start by removing variables with high p-values (greater than 0.05) first, so I removed coarse aggregate, fine aggregate, and water to cement ratio, which gave me my model2_2. This resulted in a model with VIFs lower than 5 for the remaining variables, and low p-values. In addition, the R^2 for this model was closer to the initial R^2 -- 0.600, which means model2_2 was a better fit ot the training data than model4. Additionally, the OSR^2 for model2_2 was 0.634, making it higher than the OSR^2 for model4. 

Thus, model2_2 is a better fit for the training and test data than model4 because of its higher R^2 and OSR^2 values respectively. This means that for this dataset, it was more appropriate to remove variables with high p-values rather than starting with the VIFs.